# Practice notebook for hypothesis tests using NHANES data

This notebook will give you the opportunity to perform some hypothesis tests with the NHANES data that are similar to
what was done in the week 3 case study notebook.

You can enter your code into the cells that say "enter your code here", and you can type responses to the questions into the cells that say "Type Markdown and Latex".

Note that most of the code that you will need to write below is very similar to code that appears in the case study notebook.  You will need to edit code from that notebook in small ways to adapt it to the prompts below.

To get started, we will use the same module imports and read the data in the same way as we did in the case study:

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

da = pd.read_csv("nhanes_2015_2016.csv")

## Question 1

Conduct a hypothesis test (at the 0.05 level) for the null hypothesis that the proportion of women who smoke is equal to the proportion of men who smoke.

In [21]:
df_smk = da[['SMQ020','RIAGENDR']].copy()
df_smk['Gender'] = np.where(df_smk['RIAGENDR']==1,'Male','Female')
df_smk['Smoker'] = np.where(df_smk['SMQ020']==1,1,0)
df_smk.drop(columns=['SMQ020','RIAGENDR'],inplace=True)
df_smk.head()

,Gender,Smoker
0,Male,1
1,Male,1
2,Male,1
3,Female,0
4,Female,0


In [49]:
ct = pd.crosstab(df_smk['Smoker'],df_smk['Gender'])

n_female = ct.loc[0,'Female']+ct.loc[1,'Female']
p_female = ct.loc[1,'Female']/n_female

n_male = ct.loc[0,'Male']+ct.loc[1,'Male']
p_male = ct.loc[1,'Male']/n_male

print('%.3f'%p_female,n_female)
print('%.3f'%p_male,n_male)
print(n_male+n_female)

ct

0.304 2976
0.512 2759
5735


Gender,Female,Male
Smoker,,
0,2070,1346
1,906,1413


In [48]:
df_smk_males = df_smk[df_smk['Gender']=='Male']
df_smk_females = df_smk[df_smk['Gender']=='Female']

sm.stats.ttest_ind(df_smk_males['Smoker'], df_smk_females['Smoker'])

(16.38166733131429, 5.561943460353374e-59, 5733.0)

__Q1b.__ Construct three 95% confidence intervals: one for the proportion of women who smoke, one for the proportion of men who smoke, and one for the difference in the rates of smoking between women and men.

In [58]:
print('CI for the proportion of males who smoke: \t',sm.stats.proportion_confint ( df_smk_males['Smoker'].sum() , len(df_smk_males['Smoker']) ))
print('CI for the proportion of females who smoke: \t',sm.stats.proportion_confint ( df_smk_females['Smoker'].sum() , len(df_smk_females['Smoker']) ))

CI for the proportion of males who smoke: 	 (0.49349056386167417, 0.5307935970661982)
CI for the proportion of females who smoke: 	 (0.2879026244757051, 0.3209683432662304)


In [63]:
meanCI_male = sm.stats.DescrStatsW(df_smk_males['Smoker'])
meanCI_female = sm.stats.DescrStatsW(df_smk_females['Smoker'])

print(meanCI_male.zconfint_mean())
print(meanCI_female.zconfint_mean())
print(sm.stats.CompareMeans(meanCI_male,meanCI_female).zconfint_diff())

(0.49348718281993476, 0.5307969781079376)
(0.28789984607733315, 0.32097112166460234)
(0.18285580207735627, 0.23255739110858056)


__Q1c.__ Comment on any ways in which the confidence intervals that you found in part b reinforce, contradict, or add support to the hypothesis test conducted in part a.

## Question 2

Partition the population into two groups based on whether a person has graduated college or not, using the educational attainment variable [DMDEDUC2](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDEDUC2).  Then conduct a test of the null hypothesis that the average heights (in centimeters) of the two groups are equal.  Next, convert the heights from centimeters to inches, and conduct a test of the null hypothesis that the average heights (in inches) of the two groups are equal.

In [90]:
df_educ = da.copy()
df_educ['CompletedCollege'] = np.where(df_educ['DMDEDUC2']==5,1,0)
df_educ = df_educ[['BMXHT','CompletedCollege']]
df_educ.rename(columns={'BMXHT':'Height'},inplace=True)
df_educ.dropna(inplace=True)
df_educ.head()

,Height,CompletedCollege
0,184.5,1
1,171.4,0
2,170.1,0
3,160.9,1
4,164.9,0


In [91]:
df_educ.groupby('CompletedCollege').mean()

,Height
CompletedCollege,
0,165.605442
1,167.855351


In [94]:
print(sm.stats.ztest(df_educ[df_educ['CompletedCollege']==1]['Height'], df_educ[df_educ['CompletedCollege']==0]['Height'])) # prints test statistic, p-value
print(sm.stats.ttest_ind(df_educ[df_educ['CompletedCollege']==1]['Height'], df_educ[df_educ['CompletedCollege']==0]['Height']))

(7.2007656698012505, 5.987537169067422e-13)
(7.200765669801341, 6.766583030851776e-13, 5671.0)


__Q2a.__ Based on the analysis performed here, are you confident that people who graduated from college have a different average height compared to people who did not graduate from college?

__Q2b:__ How do the results obtained using the heights expressed in inches compare to the results obtained using the heights expressed in centimeters?

## Question 3

Conduct a hypothesis test of the null hypothesis that the average BMI for men between 30 and 40 is equal to the average BMI for men between 50 and 60.  Then carry out this test again after log transforming the BMI values.

In [125]:
df_men = da[['BMXBMI','RIDAGEYR','RIAGENDR']].copy()
df_men.dropna(inplace=True)
df_men = df_men[df_men['RIAGENDR']==1]
df_men['agegrp'] = np.select(
    [(df_men['RIDAGEYR']>=30) & (df_men['RIDAGEYR']<=40),
      (df_men['RIDAGEYR']>=50) & (df_men['RIDAGEYR']<=60)],
    ['30-40','50-60'],
     default=np.nan)
df_men = df_men[df_men['agegrp']!='nan']
df_men.drop(columns=['RIDAGEYR','RIAGENDR'],inplace=True)
df_men

,BMXBMI,agegrp
1,30.8,50-60
9,33.6,50-60
24,25.0,50-60
28,30.8,50-60
32,24.4,50-60
...,...,...
5720,24.2,30-40
5725,30.9,30-40
5726,32.0,50-60
5728,32.9,30-40


In [126]:
df_men.groupby('agegrp').mean()

,BMXBMI
agegrp,
30-40,29.61373
50-60,29.25000


In [127]:
print(sm.stats.ztest(df_men[df_men['agegrp']=='30-40']['BMXBMI'], df_men[df_men['agegrp']=='50-60']['BMXBMI'])) # prints test statistic, p-value
print(sm.stats.ttest_ind(df_men[df_men['agegrp']=='30-40']['BMXBMI'], df_men[df_men['agegrp']=='50-60']['BMXBMI']))

(0.8984008016755222, 0.36897190924214873)
(0.898400801675522, 0.36919303123271296, 978.0)


__Q3a.__ How would you characterize the evidence that mean BMI differs between these age bands, and how would you characterize the evidence that mean log BMI differs between these age bands?

## Question 4

Suppose we wish to compare the mean BMI between college graduates and people who have not graduated from college, focusing on women between the ages of 30 and 40.  First, consider the variance of BMI within each of these subpopulations using graphical techniques, and through the estimated subpopulation variances.  Then, calculate pooled and unpooled estimates of the standard error for the difference between the mean BMI in the two populations being compared.  Finally, test the null hypothesis that the two population means are equal, using each of the two different standard errors.

In [ ]:
# insert your code here

__Q4a.__ Comment on the strength of evidence against the null hypothesis that these two populations have equal mean BMI.

__Q4b.__ Comment on the degree to which the two populations have different variances, and on the extent to which the results using different approaches to estimating the standard error of the mean difference give divergent results.

## Question 5

Conduct a test of the null hypothesis that the first and second diastolic blood pressure measurements within a subject have the same mean values.

In [ ]:
# insert your code here

__Q5a.__ Briefly describe your findings for an audience that is not familiar with statistical hypothesis testing.

__Q5b.__ Pretend that the first and second diastolic blood pressure measurements were taken on different people.  Modfify the analysis above as appropriate for this setting.

In [ ]:
# insert your code here

__Q5c.__ Briefly describe how the approaches used and the results obtained in the preceeding two parts of the question differ.